In [1]:
import pandas as pd
from lib import create_portfolio
import numpy as np
import scipy.optimize as sp

In [2]:
RISK_FREE = 0

In [3]:
data = pd.read_csv("data.csv", index_col=0, header=[0, 1])

In [4]:
# Using Daily Change in price
log_returns = np.log(data["Close"] / data["Close"].shift(1))

# Create Co-variance table
covariance_table = log_returns.cov() * np.sqrt(252)

In [5]:
weights = np.array([1./len(log_returns.columns)] * len(log_returns.columns))

create_portfolio(log_returns, weights, log_returns.cov())

c:\Users\olive\Documents\GitHub\Modern-Portfolio-Theory\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


{'return': Ticker
 AAPL    44.477214
 AMD     32.407002
 KO      10.049685
 MS      37.570371
 MSFT    36.333657
 TSLA    95.208454
 TSM     46.346689
 dtype: float64,
 'std': np.float64(0.2903365115438384)}

In [6]:
x_0 = np.array([0.001] * len(log_returns.columns))
args = (log_returns.mean(), covariance_table)
contraints = {"type": "eq", "fun": lambda x: np.sum(x) - 1}
bound = (0,1)
bounds = [bound for x in range(len(log_returns.columns))]

In [11]:
def risk_target(weights, returns, co_variance_table):
    return create_portfolio(returns, weights, co_variance_table)["std"]


def return_target(weights, returns, co_variance_table):
    return -create_portfolio(returns, weights, co_variance_table)["return"]


def sharpe_target(weights, returns, co_variance_table):
    portfolio = create_portfolio(returns, weights, co_variance_table)
    sharpe = (portfolio["return"] - RISK_FREE) / (portfolio["std"])
    return -sharpe

In [ ]:
lowest_risk = sp.minimize(risk_target, x_0, args=args, constraints=contraints, bounds=bounds)
max_return = sp.minimize(return_target, x_0, args=args, constraints=contraints, bounds=bounds)
max_sharpe = sp.minimize(sharpe_target, x_0, args=args, constraints=contraints, bounds=bounds)

In [17]:
{x: round(y,4) for x,y in zip(log_returns.columns, lowest_risk.x)} 

{'AAPL': np.float64(0.0179),
 'AMD': np.float64(0.0),
 'KO': np.float64(0.7679),
 'MS': np.float64(0.0),
 'MSFT': np.float64(0.0787),
 'TSLA': np.float64(0.0),
 'TSM': np.float64(0.1354)}

In [18]:
create_portfolio(log_returns.mean(), lowest_risk.x, covariance_table)

{'return': np.float64(0.0982980284711378),
 'std': np.float64(0.7818239297986908)}